In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'


import cv2
import imghdr
import csv
import numpy as np
import pathlib
from tensorflow import keras
from keras.models import Sequential
from keras import optimizers
from keras.layers import Convolution2D, MaxPooling2D, Dropout, Flatten, Dense, Reshape, LSTM, BatchNormalization
from keras.optimizers import SGD, RMSprop, Adam
from keras import backend as K
from keras.constraints import maxnorm
import tensorflow as tf
from scipy import io as spio
import idx2numpy  # sudo pip3 install idx2numpy
from matplotlib import pyplot as plt
from typing import *
import time
from pdf2image import convert_from_path
import pytesseract


def cnn_print_digit(d):
    print(d.shape)
    for x in range(28):
        s = ""
        for y in range(28):
            s += "{0:.1f} ".format(d[28*y + x])
        print(s)


def cnn_print_digit_2d(d):
    print(d.shape)
    for y in range(d.shape[0]):
        s = ""
        for x in range(d.shape[1]):
            s += "{0:.1f} ".format(d[x][y])
        print(s)


emnist_labels = [48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122]


def emnist_model():
    model = Sequential()
    model.add(Convolution2D(filters=32, kernel_size=(3, 3), padding='valid', input_shape=(28, 28, 1), activation='relu'))
    model.add(Convolution2D(filters=64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(len(emnist_labels), activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])
    return model

def emnist_model2():
    model = Sequential()
    # In Keras there are two options for padding: same or valid. Same means we pad with the number on the edge and valid means no padding.
    model.add(Convolution2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(28, 28, 1)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Convolution2D(64, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Convolution2D(128, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    # model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    # model.add(MaxPooling2D((2, 2)))
    ## model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(len(emnist_labels), activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])
    return model

def emnist_model3():
    model = Sequential()
    model.add(Convolution2D(filters=32, kernel_size=(3, 3), padding='same', input_shape=(28, 28, 1), activation='relu'))
    model.add(Convolution2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Convolution2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Convolution2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(len(emnist_labels), activation="softmax"))
    model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0), metrics=['accuracy'])
    return model


def emnist_train(model):
    t_start = time.time()

    emnist_path = './data/'
    X_train = idx2numpy.convert_from_file(emnist_path + 'emnist-byclass-train-images-idx3-ubyte')
    y_train = idx2numpy.convert_from_file(emnist_path + 'emnist-byclass-train-labels-idx1-ubyte')

    X_test = idx2numpy.convert_from_file(emnist_path + 'emnist-byclass-test-images-idx3-ubyte')
    y_test = idx2numpy.convert_from_file(emnist_path + 'emnist-byclass-test-labels-idx1-ubyte')

    X_train = np.reshape(X_train, (X_train.shape[0], 28, 28, 1))
    X_test = np.reshape(X_test, (X_test.shape[0], 28, 28, 1))

    print(X_train.shape, y_train.shape, X_test.shape, y_test.shape, len(emnist_labels))

    # Test:
    k = 1
    X_train = X_train[:X_train.shape[0] // k]
    y_train = y_train[:y_train.shape[0] // k]
    X_test = X_test[:X_test.shape[0] // k]
    y_test = y_test[:y_test.shape[0] // k]

    # Normalize
    X_train = X_train.astype(np.float32)
    X_train /= 255.0
    X_test = X_test.astype(np.float32)
    X_test /= 255.0

    x_train_cat = keras.utils.to_categorical(y_train, len(emnist_labels))
    y_test_cat = keras.utils.to_categorical(y_test, len(emnist_labels))

    # Set a learning rate reduction
    learning_rate_reduction = keras.callbacks.ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=1, factor=0.5, min_lr=0.00001)

    # Required for learning_rate_reduction:
    keras.backend.get_session().run(tf.global_variables_initializer())

    model.fit(X_train, x_train_cat, validation_data=(X_test, y_test_cat), callbacks=[learning_rate_reduction], batch_size=64, epochs=30)
    print("Training done, dT:", time.time() - t_start)


def emnist_predict(model, image_file):
    img = keras.preprocessing.image.load_img(image_file, target_size=(28, 28), color_mode='grayscale')
    emnist_predict_img(model, img)


def emnist_predict_img(model, img):
    img_arr = np.expand_dims(img, axis=0)
    img_arr = 1 - img_arr/255.0
    img_arr[0] = np.rot90(img_arr[0], 3)
    img_arr[0] = np.fliplr(img_arr[0])
    img_arr = img_arr.reshape((1, 28, 28, 1))

    result = model.predict_classes([img_arr])
    return chr(emnist_labels[result[0]])


def letters_extract(image_file: str, out_size=28):
    img = cv2.imread(image_file)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, \
                                   cv2.THRESH_BINARY, 15, 3)
    img_erode = cv2.erode(thresh, np.ones((2,3), np.uint8), iterations=1)

    # Get contours
    contours, hierarchy = cv2.findContours(img_erode, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

    output = img.copy()

    letters = []
    for idx, contour in enumerate(contours):
        (x, y, w, h) = cv2.boundingRect(contour)
        if h > 10:
            if hierarchy[0][idx][3] == 0:
                cv2.rectangle(output, (x, y), (x + w, y + h), (70, 0, 0), 1)
                letter_crop = gray[y:y + h, x:x + w]

                # Resize letter canvas to square
                size_max = max(w, h)
                letter_square = 255 * np.ones(shape=[size_max, size_max], dtype=np.uint8)
                if w > h:
                    # Enlarge image top-bottom
                    # ------
                    # ======
                    # ------
                    y_pos = size_max//2 - h//2
                    letter_square[y_pos:y_pos + h, 0:w] = letter_crop
                elif w < h:
                    # Enlarge image left-right
                    # --||--
                    x_pos = size_max//2 - w//2
                    letter_square[0:h, x_pos:x_pos + w] = letter_crop
                else:
                    letter_square = letter_crop

                # Resize letter to 28x28 and add letter and its X-coordinate
                letters.append((x, w, cv2.resize(letter_square, (out_size, out_size), interpolation=cv2.INTER_AREA)))

        # Sort array in place by X-coordinate
        letters.sort(key=lambda x: x[0], reverse=False)

    return letters


def img_to_str(model: Any, image_file: str):
    letters = letters_extract(image_file)

    s_out = ""
    for i in range(len(letters)):
        dn = letters[i+1][0] - letters[i][0] - letters[i][1] if i < len(letters) - 1 else 0
        s_out += emnist_predict_img(model, letters[i][2])
        #cv2.imshow("0", letters[i][2])
        #cv2.waitKey(0)
        if (dn > letters[i][1]/4):
            s_out += ' '
    return s_out

def  crop_pdf_to_pictures(filename: str) -> list:
    """Function to crop images from pdf file"""

    pages = convert_from_path(filename, 320)
    capture_list = []

    for i, page in enumerate(pages):
        page.save(f'./img/out_{i}.jpg', 'JPEG')
        img = cv2.imread(f'./img/out_{i}.jpg')
        if i == 0:
            crop = img[1420:1500, 1340:]
            capture_list.append(crop)
            crop = img[1640:1720, 1440:]
            capture_list.append(crop)
        if i == 1:
            crop = img[640:680, 600:]
            capture_list.append(crop)
            crop = img[740:800, 600:]
            capture_list.append(crop)
            crop = img[1132:1175, 460:]
            capture_list.append(crop)
            crop = img[1352:1415, 943:]
            capture_list.append(crop)
            crop = img[1520:1580, 943:]
            capture_list.append(crop)
        if i == 2:
            crop = img[780:848, 1620:]
            capture_list.append(crop)
            crop = img[852:900, 1620:]
            capture_list.append(crop)
            crop = img[960:1020, 1416:]
            capture_list.append(crop)
            crop = img[1012:1070, 1416:]
            capture_list.append(crop)
            crop = img[1128:1184, 1728:]
            capture_list.append(crop)
            crop = img[1188:1240, 1728:]
            capture_list.append(crop)
            crop = img[1680:1728, 720:]
            capture_list.append(crop)
            crop = img[1780:1840, 1092:]
            capture_list.append(crop)
            crop = img[1880:1940, 1092:]
            capture_list.append(crop)
        if i == 3:
            crop = img[360:420, 760:]
            capture_list.append(crop)
            crop = img[710:770, 952:]
            capture_list.append(crop)
            crop = img[910:970, 1100:]
            capture_list.append(crop)
            crop = img[1085:1145, 1180:]
            capture_list.append(crop)
            crop = img[1885:1945, 1360:]
            capture_list.append(crop)
            crop = img[2305:2365, 652:]
            capture_list.append(crop)
            crop = img[2425:2485, 648:]
            capture_list.append(crop)

    return capture_list

if __name__ == "__main__":
    """Remove # for training text recognition models"""
    
#     #############################################
#     model = emnist_model()
#     emnist_train(model)
#     model.save('emnist_letters.h5')
#     ##############################################

    """loading the model trained in an hour"""
    model = keras.models.load_model('emnist_letters.h5')
    """loading the model trained in an 10 hour"""
#     model = keras.models.load_model('emnist_letters_new.h5')
    """Found pdf files"""
    pdf_files = os.listdir('pdf')

    all_data = []
    for pdf_file in pdf_files:

        """cut pdf pictures"""
        captures = crop_pdf_to_pictures(f'./pdf/{pdf_file}')
        list_values = []
        for i, capture in enumerate(captures):
            cv2.imshow('from_file', capture)
            cv2.waitKey(0)
            cv2.imwrite('./img/test.png', capture)
            if i in (1,2,3,13,17,18,19,20,21,22):
                s_out = pytesseract.image_to_string(capture)
            else:
                s_out = img_to_str(model, "./img/test.png")
            list_values.append(s_out)
            print(i, s_out)

        all_data.append(list_values)

    print (all_data)

    with open('result_1_hour.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["date", "name", "Base Currency", "Eligible Currency", "Delivery Amount", "Return Amount",
                         "Credit Support Amount", "Independent Amount (Party A)","Independent Amount (Party B)",
                         "Threshold (Party A)","Threshold (Party B)", "Minimum Transfer Amount (Party A)", "Minimum Transfer Amount (Party B)",
                         "Valuation Agent", "Valuation Date", "Exchange Date", "Resolution Time", "Value", "Alternative",
                         "Transfer of Interest Amount", "Addresses for Transfers(Party A)  ","Addresses for Transfers(Party B)  "])
        for c in all_data:
            writer.writerow(c)

Using TensorFlow backend.


0 03 05 2011
1 Smaug International
2 KZT
3 XAU
4 1750 000 00
5 875007000 00
6 I785030W00
7 210 0W00
8 50 000 00
9 17000 00
10 
11 1 X07000 WM
12 I 1270 000 WM
13 CREDIT SUISSE
14 IW
15 
16 
17 09:00 AM
18 
19 
20 inerest(@smaug.com
21 no_name_bank@riddle.com
22 - Smaug@smaug.com
0 03 05 2011
1 Iron Bank
2 EUR
3 JPY
4 500 000 00
5 5000W0 00
6 18503W0 00
7 I 03000 00
8 150 000 00
9 00 00
10 00 00
11 50 W0 00WSD
12 00 000 00USD
13 GOLDMAN SACHS
14 ItT1I
15 
16 
17 10:00 AM
18 
19 
20 inerest@ ironbank.com
21 no_name_bank@riddle.com
22 collateral@ironbank.com
0 13 04 2010
1 Bank Gringotts
2 USD
3 CHF
4 2 500 000 00
5 175007000 00
6 7503W0 00
7 300 W0 00
8 11250 000 00
9 0
10 1 000 00
11 250 000 00M
12 
13 JP MORGAN CHASE
14 It7
15 
16 
17 11:00 AM
18 
19 
20 inerest@gringotts.com
21 no_name_bank@riddle.com
22 middleoffice@gringotts.com
[['03 05 2011', 'Smaug International', 'KZT', 'XAU', '1750 000 00', '875007000 00', 'I785030W00', '210 0W00', '50 000 00', '17000 00', '', '1 X07000 WM', 'I

In [2]:
if __name__ == "__main__":
    
    """loading the model trained in an 10 hour"""
    
    model = keras.models.load_model('emnist_letters_new.h5')
    """Found pdf files"""
    pdf_files = os.listdir('pdf')

    all_data = []
    for pdf_file in pdf_files:

        """cut pdf pictures"""
        captures = crop_pdf_to_pictures(f'./pdf/{pdf_file}')
        list_values = []
        for i, capture in enumerate(captures):
            cv2.imshow('from_file', capture)
            cv2.waitKey(0)
            cv2.imwrite('./img/test.png', capture)
            if i in (1,2,3,13,17,18,19,20,21,22):
                s_out = pytesseract.image_to_string(capture)
            else:
                s_out = img_to_str(model, "./img/test.png")
            list_values.append(s_out)
            print(i, s_out)

        all_data.append(list_values)

    print (all_data)

    with open('result_10_hours.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["date", "name", "Base Currency", "Eligible Currency", "Delivery Amount", "Return Amount",
                         "Credit Support Amount", "Independent Amount (Party A)","Independent Amount (Party B)",
                         "Threshold (Party A)","Threshold (Party B)", "Minimum Transfer Amount (Party A)", "Minimum Transfer Amount (Party B)",
                         "Valuation Agent", "Valuation Date", "Exchange Date", "Resolution Time", "Value", "Alternative",
                         "Transfer of Interest Amount", "Addresses for Transfers(Party A)  ","Addresses for Transfers(Party B)  "])
        for c in all_data:
            writer.writerow(c)

0 03 05 20I1
1 Smaug International
2 KZT
3 XAU
4 1750 0U0 00
5 875U070U0 00
6 1785030W0U
7 210 0W00
8 50 000 00
9 170UU 00
10 
11 1 E07000 Mm
12 l i270 0UU Mm
13 CREDIT SUISSE
14 GM
15 
16 
17 09:00 AM
18 
19 
20 inerest(@smaug.com
21 no_name_bank@riddle.com
22 - Smaug@smaug.com
0 03 05 20I1
1 Iron Bank
2 EUR
3 JPY
4 500 0U0 00
5 5003W0 00
6 i8503W0 00
7 1 03000 00
8 150 000 00
9 U0 00
10 0U U0
11 50 W0 00USD
12 00 0U0 0UUSD
13 GOLDMAN SACHS
14 GtwHI
15 
16 
17 10:00 AM
18 
19 
20 inerest@ ironbank.com
21 no_name_bank@riddle.com
22 collateral@ironbank.com
0 13 04 2010
1 Bank Gringotts
2 USD
3 CHF
4 2 500 0U0 00
5 175U070U0 00
6 7503W0 00
7 300 W0 0U
8 HH250 000 00
9 U
10 1 000 00
11 250 U00 00m
12 
13 JP MORGAN CHASE
14 GF2
15 
16 
17 11:00 AM
18 
19 
20 inerest@gringotts.com
21 no_name_bank@riddle.com
22 middleoffice@gringotts.com
[['03 05 20I1', 'Smaug International', 'KZT', 'XAU', '1750 0U0 00', '875U070U0 00', '1785030W0U', '210 0W00', '50 000 00', '170UU 00', '', '1 E07000 Mm', 'l

In [3]:
if __name__ == "__main__":
    
    """pyressaract perfomance"""

    """Found pdf files"""
    pdf_files = os.listdir('pdf')

    all_data = []
    for pdf_file in pdf_files:

        """cut pdf pictures"""
        captures = crop_pdf_to_pictures(f'./pdf/{pdf_file}')
        list_values = []
        for i, capture in enumerate(captures):
            cv2.imshow('from_file', capture)
            cv2.waitKey(0)
            cv2.imwrite('./img/test.png', capture)
            s_out = pytesseract.image_to_string(capture)
            list_values.append(s_out)
            print(i, s_out)

        all_data.append(list_values)

    print (all_data)

    with open('result_pytessaract.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["date", "name", "Base Currency", "Eligible Currency", "Delivery Amount", "Return Amount",
                         "Credit Support Amount", "Independent Amount (Party A)","Independent Amount (Party B)",
                         "Threshold (Party A)","Threshold (Party B)", "Minimum Transfer Amount (Party A)", "Minimum Transfer Amount (Party B)",
                         "Valuation Agent", "Valuation Date", "Exchange Date", "Resolution Time", "Value", "Alternative",
                         "Transfer of Interest Amount", "Addresses for Transfers(Party A)  ","Addresses for Transfers(Party B)  "])
        for c in all_data:
            writer.writerow(c)

0 03.05.2011
1 Smaug International
2 KZT
3 XAU
4 
5 8,500,000.00
6 1,850,000.00
7 210,000.00
8 50,000.00
9 1,000.00
10 1,000.00
11 1,250,000.00 KZT
12 1,270,000.00 KZT
13 CREDIT SUISSE
14 
15 
16 T+l
17 09:00 AM
18 
19 
20 inerest(@smaug.com
21 no_name_bank@riddle.com
22 - Smaug@smaug.com
0 03.05.2011
1 Iron Bank
2 EUR
3 JPY
4 
5 500,000.00
6 850,000.00
7 1 0,000.00
8 150,000.00
9 00.00
10 00.00
11 50,000.00 USD
12 200,000.00 USD
13 GOLDMAN SACHS
14 T+
15 
16 T+2
17 10:00 AM
18 
19 
20 inerest@ ironbank.com
21 no_name_bank@riddle.com
22 collateral@ironbank.com
0 13.04.2010
1 Bank Gringotts
2 USD
3 CHF
4 
5 1,500,000.00
6 750,000.00
7 300,000.00
8 250,000.00
9 
10 1,000.00
11 250,000.00 EUR
12 270,000.00 EUR
13 JP MORGAN CHASE
14 TH
15 
16 TB
17 11:00 AM
18 
19 
20 inerest@gringotts.com
21 no_name_bank@riddle.com
22 middleoffice@gringotts.com
[['03.05.2011', 'Smaug International', 'KZT', 'XAU', '', '8,500,000.00', '1,850,000.00', '210,000.00', '50,000.00', '1,000.00', '1,000.00', '1,250

In [4]:
import pandas as pd
df1 = pd.read_csv("./result_1_hour.csv")
df2 = pd.read_csv("./result_10_hours.csv")
df3 =pd.read_csv("./result_pytessaract.csv")


In [5]:
df1

,date,name,Base Currency,Eligible Currency,Delivery Amount,Return Amount,Credit Support Amount,Independent Amount (Party A),Independent Amount (Party B),Threshold (Party A),...,Minimum Transfer Amount (Party B),Valuation Agent,Valuation Date,Exchange Date,Resolution Time,Value,Alternative,Transfer of Interest Amount,Addresses for Transfers(Party A),Addresses for Transfers(Party B)
03 05 2011,Smaug International,KZT,XAU,1750 000 00,875007000 00,I785030W00,210 0W00,50 000 00,17000 00,NaN,...,CREDIT SUISSE,IW,NaN,NaN,09:00 AM,NaN,NaN,inerest(@smaug.com,no_name_bank@riddle.com,- Smaug@smaug.com
03 05 2011,Iron Bank,EUR,JPY,500 000 00,5000W0 00,18503W0 00,I 03000 00,150 000 00,00 00,00 00,...,GOLDMAN SACHS,ItT1I,NaN,NaN,10:00 AM,NaN,NaN,inerest@ ironbank.com,no_name_bank@riddle.com,collateral@ironbank.com
13 04 2010,Bank Gringotts,USD,CHF,2 500 000 00,175007000 00,7503W0 00,300 W0 00,11250 000 00,0,1 000 00,...,JP MORGAN CHASE,It7,NaN,NaN,11:00 AM,NaN,NaN,inerest@gringotts.com,no_name_bank@riddle.com,middleoffice@gringotts.com


In [6]:
df2

,date,name,Base Currency,Eligible Currency,Delivery Amount,Return Amount,Credit Support Amount,Independent Amount (Party A),Independent Amount (Party B),Threshold (Party A),...,Minimum Transfer Amount (Party B),Valuation Agent,Valuation Date,Exchange Date,Resolution Time,Value,Alternative,Transfer of Interest Amount,Addresses for Transfers(Party A),Addresses for Transfers(Party B)
03 05 20I1,Smaug International,KZT,XAU,1750 0U0 00,875U070U0 00,1785030W0U,210 0W00,50 000 00,170UU 00,NaN,...,CREDIT SUISSE,GM,NaN,NaN,09:00 AM,NaN,NaN,inerest(@smaug.com,no_name_bank@riddle.com,- Smaug@smaug.com
03 05 20I1,Iron Bank,EUR,JPY,500 0U0 00,5003W0 00,i8503W0 00,1 03000 00,150 000 00,U0 00,0U U0,...,GOLDMAN SACHS,GtwHI,NaN,NaN,10:00 AM,NaN,NaN,inerest@ ironbank.com,no_name_bank@riddle.com,collateral@ironbank.com
13 04 2010,Bank Gringotts,USD,CHF,2 500 0U0 00,175U070U0 00,7503W0 00,300 W0 0U,HH250 000 00,U,1 000 00,...,JP MORGAN CHASE,GF2,NaN,NaN,11:00 AM,NaN,NaN,inerest@gringotts.com,no_name_bank@riddle.com,middleoffice@gringotts.com


In [7]:
df3

,date,name,Base Currency,Eligible Currency,Delivery Amount,Return Amount,Credit Support Amount,Independent Amount (Party A),Independent Amount (Party B),Threshold (Party A),...,Minimum Transfer Amount (Party B),Valuation Agent,Valuation Date,Exchange Date,Resolution Time,Value,Alternative,Transfer of Interest Amount,Addresses for Transfers(Party A),Addresses for Transfers(Party B)
03.05.2011,Smaug International,KZT,XAU,NaN,"8,500,000.00","1,850,000.00","210,000.00","50,000.00","1,000.00","1,000.00",...,CREDIT SUISSE,NaN,NaN,T+l,09:00 AM,NaN,NaN,inerest(@smaug.com,no_name_bank@riddle.com,- Smaug@smaug.com
03.05.2011,Iron Bank,EUR,JPY,NaN,"500,000.00","850,000.00","1 0,000.00","150,000.00",00.00,00.00,...,GOLDMAN SACHS,T+,NaN,T+2,10:00 AM,NaN,NaN,inerest@ ironbank.com,no_name_bank@riddle.com,collateral@ironbank.com
13.04.2010,Bank Gringotts,USD,CHF,NaN,"1,500,000.00","750,000.00","300,000.00","250,000.00",NaN,"1,000.00",...,JP MORGAN CHASE,TH,NaN,TB,11:00 AM,NaN,NaN,inerest@gringotts.com,no_name_bank@riddle.com,middleoffice@gringotts.com
